In [2]:
__author__ = 'swe03'

import argparse

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import matplotlib.pyplot as pplt

from decimal import *

%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 15, 6

from statsmodels.graphics.api import qqplot

from scipy import stats

desired_width = 400
pd.set_option('display.width',desired_width)

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [3]:
import datetime
from dateutil.relativedelta import relativedelta
import seaborn as sns
import statsmodels.api as sm  
import statsmodels.tsa as tsa
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [4]:
#!pip install pandasql
from pandasql import PandaSQL 
pdsql = PandaSQL()

##### Read the Dataframe created from the Iterate-and-Create code  

In [ ]:
ent_calc_df1

In [ ]:
### Copied this from Iterate and Create for Out Entropy but was not used(i.e., commented out in the program).  

ent_calc_df1.reset_index(inplace=True)
#ent_calc_df1

In [ ]:
ent_calc_df1['flStDttmEst'] = pd.to_datetime(ent_calc_df1['flStDttmEst'] )
#ent_calc_df1

In [ ]:
### Create a new index that has the correct dtype='datetime64[ns]
ent_calc_df1 = ent_calc_df1.set_index('flStDttmEst')
ent_calc_df1.sort_index(inplace=True)

In [ ]:
ent_calc_df1.index
#ent_calc_df1

In [ ]:
## Specify the SARIMAX model
## Default for the CI is 95%.  Set in the Alpha parameter for conf_int function
mod = sm.tsa.statespace.SARIMAX(ent_calc_df1['out_degree_entropy'], 
         order=(3,1,0), seasonal_order=(3,0,0,12))
mod1 = sm.tsa.statespace.SARIMAX(ent_calc_df1['out_degree_entropy'], 
      order=(8,0,0), seasonal_order=(2,0,0,6))
mod2 = sm.tsa.statespace.SARIMAX(ent_calc_df1['out_degree_entropy'], 
     trend='n', order=(1,0,0), seasonal_order=(2,0,0,24))
results = mod.fit()
### Probably need to write this out to the BI Data Model and NOT print it here.....
##print(results.summary())                                

## Need to write out to BI Data Model some of what is below, for example, Post Model Goodness of Fit Assessments

In [ ]:
fig = results.plot_diagnostics()

In [ ]:
## Check to see if Residuals are still correlated
## Values are between 0 and 4.  Values close to 2: no Serial Correlation. Close to 0: Pos Corr. Close to 4: Neg Corr.
sm.stats.durbin_watson(results.resid.values)

In [ ]:
## Check to see if the distributionb of Residuals are Normally Distributed (i.e., This is undesirable)
##If the p-val is very small, it means it is unlikely that the data came from a normal distribution
resid1 = results.resid
stats.normaltest(resid1)

In [ ]:
# Graph the acf and pacf for the Residuals
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(resid1.values.squeeze(), lags=10, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(resid1, lags=10, ax=ax2)

In [ ]:
# Box-Pierce Q statistic tests the Null Ho that *ALL* correlations up to Lag K are equal to Zero.  This is not the 
# same as the correlogram above.
r,q,p = sm.tsa.acf(resid1.values.squeeze(), qstat=True)
data = np.c_[range(1,41), r[1:], q, p]
table = pd.DataFrame(data, columns=['lag', "AC", "Q", "Prob(>Q)"])
print(table.set_index('lag'))

#### Prediction and Forecasting Section

In [ ]:
# This is just a SARIMAX Results Wrapper.  It collects the model parameters for later use.
res = mod.filter(results.params)

In [ ]:
### In-sample prediction and out-of-sample forecasting

### The predict function works the same as res.predict() except that exogenous variables can be 
###   included as a parameter in this function as well as more statistical functions in the predict object model 
###   like predicted_mean and predicted_results
predict = res.get_prediction()
predict.predicted_mean

In [ ]:
## Create the Confidence Intervals for the entire range
## Note that using the full range will create initial C.I.'s that are large (i.e., not enough history for tighter C.I.'s)
##   This is change the scale of the graph and is not useful for visualization below
predict_ci = predict.conf_int()
predict_ci

In [ ]:
## Create the Predicted Means for Graph #1 below
## The Predicted Means are both In-Sample Predictions as well as Out-Of-Sample Forecast
#predict_dy.predicted_mean.astype(int)
predicted_mean_1 = predict.predicted_mean.loc['2017-06-20 15:55:00':'2017-06-20 16:00:00'] 
predicted_mean_1

In [ ]:
### Create the Confidence Intervals for Graph #1 below
### This is a constrained view based on dates and is used below in the graphing
### Note:  The Start date must be the LAST Training Set data Hour
predict = res.get_prediction(start='2017-06-20 15:55:00', end='2017-06-20 16:00:00')
predict_ci_1 = predict.conf_int()
predict_ci_1